# Train CNN-RNN

Paper related https://arxiv.org/abs/1512.02595

CNN-RNN actually a small size of DeepSpeech2, renamed to CNN-RNN to avoid confusion.

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [malaya-speech/example/train-asr](https://github.com/huseinzol05/malaya-speech/tree/master/example/train-asr).
    
</div>

<div class="alert alert-warning">

This example trained on a very small dataset, do not use it for production.
    
</div>

### Model Interface

To initiate CNN-RNN,

```python
import malaya_speech.train.model.cnn_rnn as cnn_rnn

x = tf.placeholder(tf.float32, [None, None, num_features])
cnn_rnn.Model(x)
```

This model interface do not required input length because we can calculate output length using `ctc.utils.calculate_input_length_deep_speech`.

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import tensorflow as tf
from glob import glob
import malaya_speech.config
import malaya_speech.train.model.cnn_rnn as cnn_rnn
import malaya_speech.train.model.ctc as ctc
import malaya_speech.train as train
import numpy as np


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



### Load Featurizer

We prefer use `malaya_speech.tf_featurization.STTFeaturizer` for ASR.

In [4]:
config = malaya_speech.config.transducer_featurizer_config
featurizer = malaya_speech.tf_featurization.STTFeaturizer(config)
n_mels = config['num_feature_bins']
initial_learning_rate = 1e-3
max_gradient_norm = 5.0
train_steps = 1000

### Define Data Pipeline

In [5]:
def preprocess_inputs(example):
    s = featurizer.vectorize(example['waveforms'])
    mel_fbanks = tf.reshape(s, (-1, n_mels))
    example['inputs'] = mel_fbanks

    return example

In [6]:
def parse(serialized_example):

    data_fields = {
        'waveforms': tf.VarLenFeature(tf.float32),
        'targets': tf.VarLenFeature(tf.int64),
    }
    features = tf.parse_single_example(
        serialized_example, features = data_fields
    )
    for k in features.keys():
        features[k] = features[k].values

    features = preprocess_inputs(features)

    keys = list(features.keys())
    for k in keys:
        if k not in ['inputs', 'targets']:
            features.pop(k, None)

    return features


def get_dataset(path, batch_size = 2, shuffle_size = 32, thread_count = 24):
    def get():
        files = glob(path)
        dataset = tf.data.TFRecordDataset(files)
        dataset = dataset.shuffle(shuffle_size)
        dataset = dataset.map(parse, num_parallel_calls = thread_count)
        dataset = dataset.padded_batch(
            batch_size,
            padded_shapes = {
                'inputs': tf.TensorShape([None, n_mels]),
                'targets': tf.TensorShape([None]),
            },
            padding_values = {
                'inputs': tf.constant(0, dtype = tf.float32),
                'targets': tf.constant(0, dtype = tf.int64),
            },
        )
        dataset = dataset.repeat()
        return dataset

    return get

### Define Model definition

1. Use malaya_speech CTC loss,

```python
mean_error, sum_error, sum_weight = ctc.loss.ctc_loss(
    logits, targets_int32, seq_lens
)
```

2. Use malaya_speech CTC sequence accuracy,

```python
accuracy = ctc.metrics.ctc_sequence_accuracy(
    logits, targets_int32, seq_lens
)
```

This will automatically recorded in Tensorboard.

3. Use malaya_speech output length calculation,

```python
seq_lens = ctc.utils.calculate_input_length_deep_speech(
    features['inputs'], logits
)
```

4. Use malaya_speech metrics for Evaluation session,

```python
elif mode == tf.estimator.ModeKeys.EVAL:

    estimator_spec = tf.estimator.EstimatorSpec(
        mode = tf.estimator.ModeKeys.EVAL,
        loss = loss,
        eval_metric_ops = {
            'accuracy': ctc.metrics.ctc_sequence_accuracy_estimator(
                logits, targets_int32, seq_lens
            ),
            'WER': ctc.metrics.word_error_rate_estimator(
                logits, targets_int32
            ),
        },
    )
```

If we passed directly tensor values to `eval_metric_ops`, Tensorflow will throw error, required update operations. So just stick with that implementation.

In [7]:
def model_fn(features, labels, mode, params):
    model = cnn_rnn.Model(features['inputs'], training = True)
    logits = tf.layers.dense(model.logits, malaya_speech.char.VOCAB_SIZE)

    seq_lens = ctc.utils.calculate_input_length_deep_speech(
        features['inputs'], logits
    )
    targets_int32 = tf.cast(features['targets'], tf.int32)

    mean_error, sum_error, sum_weight = ctc.loss.ctc_loss(
        logits, targets_int32, seq_lens
    )

    loss = mean_error
    accuracy = ctc.metrics.ctc_sequence_accuracy(
        logits, targets_int32, seq_lens
    )

    tf.identity(loss, 'train_loss')
    tf.identity(accuracy, name = 'train_accuracy')

    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step = tf.train.get_or_create_global_step()
        learning_rate = tf.train.polynomial_decay(
            initial_learning_rate,
            global_step,
            train_steps,
            end_learning_rate = 1e-4,
            power = 1.0,
            cycle = False,
        )
        optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
        train_op = optimizer.minimize(loss, global_step = global_step)
        estimator_spec = tf.estimator.EstimatorSpec(
            mode = mode, loss = loss, train_op = train_op
        )

    elif mode == tf.estimator.ModeKeys.EVAL:

        estimator_spec = tf.estimator.EstimatorSpec(
            mode = tf.estimator.ModeKeys.EVAL,
            loss = loss,
            eval_metric_ops = {
                'accuracy': ctc.metrics.ctc_sequence_accuracy_estimator(
                    logits, targets_int32, seq_lens
                ),
                'WER': ctc.metrics.word_error_rate_estimator(
                    logits, targets_int32
                ),
            },
        )

    return estimator_spec

In [8]:
train_hooks = [
    tf.train.LoggingTensorHook(
        ['train_accuracy', 'train_loss'], every_n_iter = 1
    )
]
train_dataset = get_dataset('tolong-sebut/data/tolong-sebut-train*', batch_size = 32)
dev_dataset = get_dataset('tolong-sebut/data/tolong-sebut-dev*', batch_size = 32)

In [9]:
!rm -rf asr-cnn-rnn

### Initate training session

In [10]:
train.run_training(
    train_fn = train_dataset,
    model_fn = model_fn,
    model_dir = 'asr-cnn-rnn',
    num_gpus = 1,
    log_step = 1,
    save_checkpoint_step = 200,
    max_steps = train_steps,
    eval_fn = dev_dataset,
    train_hooks = train_hooks,
)



INFO:tensorflow:Using config: {'_model_dir': 'asr-cnn-rnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 200, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4eb5224208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation l

INFO:tensorflow:global_step/sec: 0.811702
INFO:tensorflow:train_accuracy = 0.07387863, train_loss = 52.932514 (1.232 sec)
INFO:tensorflow:loss = 52.932514, step = 14 (1.232 sec)
INFO:tensorflow:global_step/sec: 0.972217
INFO:tensorflow:train_accuracy = 0.055190537, train_loss = 53.59869 (1.029 sec)
INFO:tensorflow:loss = 53.59869, step = 15 (1.029 sec)
INFO:tensorflow:global_step/sec: 0.750876
INFO:tensorflow:train_accuracy = 0.032133676, train_loss = 61.812477 (1.332 sec)
INFO:tensorflow:loss = 61.812477, step = 16 (1.332 sec)
INFO:tensorflow:global_step/sec: 0.784571
INFO:tensorflow:train_accuracy = 0.10344828, train_loss = 53.76927 (1.275 sec)
INFO:tensorflow:loss = 53.76927, step = 17 (1.275 sec)
INFO:tensorflow:global_step/sec: 0.928189
INFO:tensorflow:train_accuracy = 0.12717536, train_loss = 46.00486 (1.077 sec)
INFO:tensorflow:loss = 46.00486, step = 18 (1.077 sec)
INFO:tensorflow:global_step/sec: 0.991931
INFO:tensorflow:train_accuracy = 0.044, train_loss = 48.279 (1.008 sec)


INFO:tensorflow:loss = 29.436512, step = 60 (1.243 sec)
INFO:tensorflow:global_step/sec: 0.97681
INFO:tensorflow:train_accuracy = 0.5193299, train_loss = 30.282953 (1.024 sec)
INFO:tensorflow:loss = 30.282953, step = 61 (1.024 sec)
INFO:tensorflow:global_step/sec: 1.05999
INFO:tensorflow:train_accuracy = 0.5184244, train_loss = 33.104588 (0.944 sec)
INFO:tensorflow:loss = 33.104588, step = 62 (0.945 sec)
INFO:tensorflow:global_step/sec: 0.865661
INFO:tensorflow:train_accuracy = 0.5483444, train_loss = 26.311527 (1.155 sec)
INFO:tensorflow:loss = 26.311527, step = 63 (1.154 sec)
INFO:tensorflow:global_step/sec: 0.986108
INFO:tensorflow:train_accuracy = 0.5399449, train_loss = 27.897413 (1.014 sec)
INFO:tensorflow:loss = 27.897413, step = 64 (1.014 sec)
INFO:tensorflow:global_step/sec: 0.978341
INFO:tensorflow:train_accuracy = 0.494311, train_loss = 32.23047 (1.022 sec)
INFO:tensorflow:loss = 32.23047, step = 65 (1.022 sec)
INFO:tensorflow:global_step/sec: 0.827942
INFO:tensorflow:train_

INFO:tensorflow:train_accuracy = 0.5349144, train_loss = 25.283634 (1.088 sec)
INFO:tensorflow:loss = 25.283634, step = 107 (1.087 sec)
INFO:tensorflow:global_step/sec: 0.942203
INFO:tensorflow:train_accuracy = 0.5241117, train_loss = 26.692331 (1.062 sec)
INFO:tensorflow:loss = 26.692331, step = 108 (1.062 sec)
INFO:tensorflow:global_step/sec: 1.02958
INFO:tensorflow:train_accuracy = 0.5168961, train_loss = 28.804419 (0.971 sec)
INFO:tensorflow:loss = 28.804419, step = 109 (0.971 sec)
INFO:tensorflow:global_step/sec: 1.04589
INFO:tensorflow:train_accuracy = 0.55989236, train_loss = 22.20596 (0.956 sec)
INFO:tensorflow:loss = 22.20596, step = 110 (0.955 sec)
INFO:tensorflow:global_step/sec: 1.07368
INFO:tensorflow:train_accuracy = 0.53978777, train_loss = 25.103392 (0.932 sec)
INFO:tensorflow:loss = 25.103392, step = 111 (0.933 sec)
INFO:tensorflow:global_step/sec: 0.715678
INFO:tensorflow:train_accuracy = 0.53737116, train_loss = 26.209068 (1.397 sec)
INFO:tensorflow:loss = 26.209068,

INFO:tensorflow:loss = 25.456984, step = 153 (1.036 sec)
INFO:tensorflow:global_step/sec: 0.988419
INFO:tensorflow:train_accuracy = 0.5388601, train_loss = 24.609137 (1.012 sec)
INFO:tensorflow:loss = 24.609137, step = 154 (1.012 sec)
INFO:tensorflow:global_step/sec: 1.01656
INFO:tensorflow:train_accuracy = 0.5410691, train_loss = 23.714384 (0.984 sec)
INFO:tensorflow:loss = 23.714384, step = 155 (0.984 sec)
INFO:tensorflow:global_step/sec: 0.948728
INFO:tensorflow:train_accuracy = 0.52902377, train_loss = 22.971615 (1.054 sec)
INFO:tensorflow:loss = 22.971615, step = 156 (1.053 sec)
INFO:tensorflow:global_step/sec: 0.976495
INFO:tensorflow:train_accuracy = 0.5567423, train_loss = 24.720615 (1.024 sec)
INFO:tensorflow:loss = 24.720615, step = 157 (1.024 sec)
INFO:tensorflow:global_step/sec: 0.794895
INFO:tensorflow:train_accuracy = 0.51294696, train_loss = 28.691193 (1.258 sec)
INFO:tensorflow:loss = 28.691193, step = 158 (1.259 sec)
INFO:tensorflow:global_step/sec: 0.695228
INFO:tenso

...

INFO:tensorflow:loss = 17.13417, step = 797 (1.093 sec)
INFO:tensorflow:global_step/sec: 1.03788
INFO:tensorflow:train_accuracy = 0.57676905, train_loss = 13.978708 (0.963 sec)
INFO:tensorflow:loss = 13.978708, step = 798 (0.962 sec)
INFO:tensorflow:Saving checkpoints for 800 into asr-cnn-rnn/model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-10-25T19:09:09Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from asr-cnn-rnn/model.ckpt-800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evalua

INFO:tensorflow:train_accuracy = 0.59485096, train_loss = 11.138513 (1.147 sec)
INFO:tensorflow:loss = 11.138513, step = 838 (1.148 sec)
INFO:tensorflow:global_step/sec: 1.05258
INFO:tensorflow:train_accuracy = 0.57310706, train_loss = 13.151548 (0.950 sec)
INFO:tensorflow:loss = 13.151548, step = 839 (0.950 sec)
INFO:tensorflow:global_step/sec: 1.00084
INFO:tensorflow:train_accuracy = 0.550447, train_loss = 17.080982 (0.999 sec)
INFO:tensorflow:loss = 17.080982, step = 840 (0.999 sec)
INFO:tensorflow:global_step/sec: 0.993625
INFO:tensorflow:train_accuracy = 0.5727637, train_loss = 13.957576 (1.006 sec)
INFO:tensorflow:loss = 13.957576, step = 841 (1.006 sec)
INFO:tensorflow:global_step/sec: 1.00855
INFO:tensorflow:train_accuracy = 0.5867209, train_loss = 12.522627 (0.992 sec)
INFO:tensorflow:loss = 12.522627, step = 842 (0.992 sec)
INFO:tensorflow:global_step/sec: 0.960574
INFO:tensorflow:train_accuracy = 0.5549872, train_loss = 14.006372 (1.041 sec)
INFO:tensorflow:loss = 14.006372,

INFO:tensorflow:loss = 12.156118, step = 884 (1.380 sec)
INFO:tensorflow:global_step/sec: 0.942727
INFO:tensorflow:train_accuracy = 0.56178343, train_loss = 14.448498 (1.058 sec)
INFO:tensorflow:loss = 14.448498, step = 885 (1.056 sec)
INFO:tensorflow:global_step/sec: 1.00715
INFO:tensorflow:train_accuracy = 0.583668, train_loss = 12.270889 (0.993 sec)
INFO:tensorflow:loss = 12.270889, step = 886 (0.993 sec)
INFO:tensorflow:global_step/sec: 1.00063
INFO:tensorflow:train_accuracy = 0.5693904, train_loss = 14.100239 (0.999 sec)
INFO:tensorflow:loss = 14.100239, step = 887 (1.000 sec)
INFO:tensorflow:global_step/sec: 1.04894
INFO:tensorflow:train_accuracy = 0.57894737, train_loss = 14.501066 (0.953 sec)
INFO:tensorflow:loss = 14.501066, step = 888 (0.953 sec)
INFO:tensorflow:global_step/sec: 0.872703
INFO:tensorflow:train_accuracy = 0.55307996, train_loss = 14.033871 (1.146 sec)
INFO:tensorflow:loss = 14.033871, step = 889 (1.147 sec)
INFO:tensorflow:global_step/sec: 0.970596
INFO:tensorf

INFO:tensorflow:global_step/sec: 0.799852
INFO:tensorflow:train_accuracy = 0.59180975, train_loss = 12.666111 (1.250 sec)
INFO:tensorflow:loss = 12.666111, step = 931 (1.251 sec)
INFO:tensorflow:global_step/sec: 0.77951
INFO:tensorflow:train_accuracy = 0.55096775, train_loss = 12.442856 (1.282 sec)
INFO:tensorflow:loss = 12.442856, step = 932 (1.281 sec)
INFO:tensorflow:global_step/sec: 0.993531
INFO:tensorflow:train_accuracy = 0.5834464, train_loss = 11.181839 (1.007 sec)
INFO:tensorflow:loss = 11.181839, step = 933 (1.007 sec)
INFO:tensorflow:global_step/sec: 1.1213
INFO:tensorflow:train_accuracy = 0.5935397, train_loss = 11.373243 (0.892 sec)
INFO:tensorflow:loss = 11.373243, step = 934 (0.891 sec)
INFO:tensorflow:global_step/sec: 1.00127
INFO:tensorflow:train_accuracy = 0.5515464, train_loss = 13.628035 (0.999 sec)
INFO:tensorflow:loss = 13.628035, step = 935 (0.999 sec)
INFO:tensorflow:global_step/sec: 0.968463
INFO:tensorflow:train_accuracy = 0.54850745, train_loss = 16.532986 (1

INFO:tensorflow:train_accuracy = 0.56041133, train_loss = 13.191481 (1.176 sec)
INFO:tensorflow:loss = 13.191481, step = 977 (1.176 sec)
INFO:tensorflow:global_step/sec: 0.992585
INFO:tensorflow:train_accuracy = 0.5751634, train_loss = 12.413516 (1.007 sec)
INFO:tensorflow:loss = 12.413516, step = 978 (1.007 sec)
INFO:tensorflow:global_step/sec: 1.02803
INFO:tensorflow:train_accuracy = 0.56916994, train_loss = 11.784748 (0.973 sec)
INFO:tensorflow:loss = 11.784748, step = 979 (0.973 sec)
INFO:tensorflow:global_step/sec: 1.01501
INFO:tensorflow:train_accuracy = 0.59517425, train_loss = 11.690419 (0.985 sec)
INFO:tensorflow:loss = 11.690419, step = 980 (0.985 sec)
INFO:tensorflow:global_step/sec: 0.953915
INFO:tensorflow:train_accuracy = 0.58431375, train_loss = 12.425091 (1.048 sec)
INFO:tensorflow:loss = 12.425091, step = 981 (1.048 sec)
INFO:tensorflow:global_step/sec: 0.914664
INFO:tensorflow:train_accuracy = 0.55569464, train_loss = 13.486433 (1.093 sec)
INFO:tensorflow:loss = 13.48

### Reference

1. Tensorflow models, https://github.com/tensorflow/models/blob/master/research/deep_speech/deep_speech_model.py